# Color with music

This project will work with the following element:
- midi keyboard as inputs
- pygame as screen, real-time updates

In [1]:
import mido
from mido import MidiFile

In [ ]:
msg = mido.Message('note_on', note=60)
msg.type

In [ ]:
msg.note

In [ ]:
msg.bytes()

In [ ]:
midi_file_path = 'SuperMarioBrothers.mid'
mid = MidiFile(midi_file_path)

In [ ]:
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

In [ ]:
for msg in mid.play():
    bytes = msg.bytes()
    print(f"length: {len(bytes)}")
    print(f"bytes: {bytes}")
    print(f"message: {msg}")
    print("================================")

## Version 0.1 (POC)
a `Message` has a few noteworthy properties that might be useful:
- `Note` (int): The midi number (could be translated to a musical note: 57 is `A3` in musical terms)
- `type` (Enum): indicates what type of input is used. Focus only on `note_on` and `note_off` for now?
- `channel` (int): will not be used in the final version because it will only have a single channel?
- `velocity` (int): indicates the velocity of note, meaning how hard the key is pushed --> alpha value?
- `time` (float): The timestamp of the message in ticks. The ticks are a unit of time in the MIDI file. might be useful

These properties need to be mapped to a few controls:
- color + alpha
- brush
- location + direction
- action

The following steps will be taken for version 0.1:
- color + alpha selection
    - colors: 7 base colors (base notes: A, B, C, D, E, F, G)
        - pallet: https://coolors.co/palette/0ad2ff-2962ff-9500ff-ff0059-ff8c00-b4e600-0fffdb
    - erasers: 5 sharp (A#, C#, D#, F#, G#). this means that some colors are not removable

- location + direction
- rendering

### color + alpha
Color mapping can be done by using the actual note. this provides two options for creating creating a color: the midi-value or the name. The name provides use with a letter (the name of the note: A, B, C, D, E, F, G), sharp or not, and a number (the octave). The midi number is just an integer that can be mapped on the name.

using the name we can map 6 base colors, with 2 extra colors if we take the sharps. and the octave could add some random saturation.

This would provide some logic to all keys making it possible to create a schema that provides some color information to users.

Otherwise, if randomness is more the goal then using a `hash` function for converting the midi-value to RGB values is the way to go. because a `hash` would create the same color of the same input, it would allow more range in colors. each key would present a new color.

for the alpha value, using `velocity` seems the best way to go. If the value cannot be mapped between 0 and 1 (or 0 and 255) then not using it for alpha is a good way.

### brush
The brush "type"/form needs to be made. It might be interesting to have different message types that change brushes over time (version 2 maybe?).

### location + direction
the location of the brush and the direction that needs to be drawn in is more difficult to map. `note_on` and `note_off`.

Given that notes tend to be short, using `dots` seems the best way to go.
for the direction, it should start at a random coordinate in a random direction, with again `velocity` the initial inertia of the stroke. the line is drawn until the `off` message is signaled or the border is hit.
the stroke after starts at the end coordinates of the previous stroke for that note with a new random direction. If the border is crossed, the end coordinate is the point of the border.

## overlapping actions
notes can be pressed at the same time. This provides one complication: keep track of the `note_on` and `note_off` message.
The on and off message both describe the same midi key so keep a `Dict` for the note (and channel?) and its status (on or off  --> `Bool`).
When the note is `on` the action is performed until the `off` message appears 

### actions
Add erase function? if so, which keys? The option to erase might be nice but also would defeat the purpose of the painting.

2 things might be fun to add:
- save picture (mail it to yourself?)
- new work (clear canvas and memory)

In [11]:
keyboard_name = "Impact GX Mini MIDI1"

In [12]:
def on_midi_message(message):
    print(f"Received MIDI message: {message}")

# Replace 'Your_MIDI_Keyboard_Name' with the actual name of your MIDI keyboard
input_port = mido.open_input(keyboard_name, callback=on_midi_message)

try:
    print(f"Listening for MIDI input from {keyboard_name}. Press Ctrl+C to exit.")
    while True:
        # The callback function on_midi_message will be called for each received MIDI message
        pass
except KeyboardInterrupt:
    print("\nExiting MIDI listener.")
finally:
    input_port.close()

Listening for MIDI input from Impact GX Mini MIDI1. Press Ctrl+C to exit.
Received MIDI message: note_on channel=0 note=53 velocity=127 time=0
Received MIDI message: note_on channel=0 note=53 velocity=0 time=0
Received MIDI message: note_on channel=0 note=57 velocity=112 time=0
Received MIDI message: note_on channel=0 note=57 velocity=0 time=0
Received MIDI message: note_on channel=0 note=57 velocity=124 time=0
Received MIDI message: note_on channel=0 note=57 velocity=0 time=0
Received MIDI message: note_on channel=0 note=57 velocity=124 time=0
Received MIDI message: note_on channel=0 note=57 velocity=0 time=0
Received MIDI message: note_on channel=0 note=57 velocity=124 time=0
Received MIDI message: note_on channel=0 note=57 velocity=0 time=0
Received MIDI message: note_on channel=0 note=57 velocity=16 time=0
Received MIDI message: note_on channel=0 note=57 velocity=0 time=0
Received MIDI message: pitchwheel channel=0 pitch=128 time=0
Received MIDI message: pitchwheel channel=0 pitch=

In [6]:
def print_available_ports():
    print("Available MIDI Input Ports:")
    for port, name in enumerate(mido.get_input_names()):
        print(f"{port}: {name}")

# Call the function to print available ports
print_available_ports()

Available MIDI Input Ports:
0: Impact GX Mini MIDI1
1: Impact GX Mini MIDI2
2: Virtuele uitgang GarageBand
